In [1]:
! pip install transformers datasets
! pip install rouge-score nltk
! pip install huggingface_hub
! pip install sentencepiece
! pip install evaluate
! pip install gdown
! pip install git-lfs

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/jovyan/.huggingface/token
Login successful


In [3]:
!git config --global credential.helper store

In [4]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [5]:
!git lfs install

!git clone https://huggingface.co/GhifSmile/mt5-base-coba-coba-coba
# Alternatively if you have a token,
# you can use it instead of your password

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any parent up to mount point /home)\nStopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).\n"
Git LFS initialized.
fatal: destination path 'mt5-base-coba-coba-coba' already exists and is not an empty directory.


In [6]:
!cd ../workspace/mt5-base-coba-coba-coba
!git config --global user.email "ismailmailghif@gmail.com"
# Tip: using the same email than for your huggingface.co account will link your commits to your profile
!git config --global user.name "GhifSmile"

In [ ]:
'''!apt install git-lfs
!git config --global user.email "ismailmailghif@gmail.com"
!git config --global user.name "GhifSmile"'''

In [7]:
from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, TFAutoModelForSeq2SeqLM
from torch.utils.data import Dataset, DataLoader
from datasets import load_metric, load_dataset,list_datasets, list_metrics
import pandas as pd
from torch import optim
from tqdm import tqdm
import torch
import re
import unicodedata

In [ ]:
import gdown

!gdown --folder --id 1-h3emdiDf-Wqlpl5mAj3ziugB__oxE9c

In [ ]:
#!tar -xzvf "/content/dataset/liputan6_data.tar.gz" -C "/content/sample_data/"     #[run this cell to extract tar.gz files]

In [ ]:
'''import os, json
import pandas as pd

# this finds our json files
path_to_json = '/content/sample_data/liputan6_data/canonical/train'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['clean_article', 'clean_summary', 'id'])'''

In [ ]:
'''# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        clean_article = json_text['clean_article']
        clean_summary = json_text['clean_summary']
        id = json_text['id']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data.loc[index] = [clean_article, clean_summary, id]

# now that we have the pertinent json data in our DataFrame let's look at it
print(jsons_data)'''

In [ ]:
#jsons_data.to_json(r'/content/drive/MyDrive/dataset/liputan6.jsonl', orient='records', lines=True)

In [8]:
data = pd.read_json('../workspace/dataset/liputan6.jsonl', lines=True)

In [9]:
data.head()

,clean_article,clean_summary,id
0,"[[Sejak, UEFA, menghapuskan, ajang, Piala, Win...","[[Meski, kini, Piala, Winner, telah, dihapus, ...",212580
1,"[[Liputan6, ., com, ,, Jakarta, :, Sekitar, 15...","[[Ratusan, pedagang, lama, Pasar, Kenari, ,, J...",37435
2,"[[Liputan6, ., com, ,, Dubai, :, Brasil, sukse...","[[Brasil, sukses, melangkah, ke, babak, final,...",252042
3,"[[Liputan6, ., com, ,, Jakarta, :, Indonesia, ...","[[ICW, mendesak, KPK, segera, menelusuri, seju...",284175
4,"[[Liputan6, ., com, ,, Temanggung, :, Pemadama...","[[Pemadaman, listrik, di, Temanggung, ,, Jaten...",253643


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193883 entries, 0 to 193882
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   clean_article  193883 non-null  object
 1   clean_summary  193883 non-null  object
 2   id             193883 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.4+ MB


In [11]:
train = data.loc[0:15000]

In [12]:
train = train.reset_index(drop=True)

In [13]:
train.head()

,clean_article,clean_summary,id
0,"[[Sejak, UEFA, menghapuskan, ajang, Piala, Win...","[[Meski, kini, Piala, Winner, telah, dihapus, ...",212580
1,"[[Liputan6, ., com, ,, Jakarta, :, Sekitar, 15...","[[Ratusan, pedagang, lama, Pasar, Kenari, ,, J...",37435
2,"[[Liputan6, ., com, ,, Dubai, :, Brasil, sukse...","[[Brasil, sukses, melangkah, ke, babak, final,...",252042
3,"[[Liputan6, ., com, ,, Jakarta, :, Indonesia, ...","[[ICW, mendesak, KPK, segera, menelusuri, seju...",284175
4,"[[Liputan6, ., com, ,, Temanggung, :, Pemadama...","[[Pemadaman, listrik, di, Temanggung, ,, Jaten...",253643


In [14]:
valid = data.loc[15001:15500]

In [15]:
valid = valid.reset_index(drop=True)

In [16]:
train.duplicated(subset='id').sum()

0

In [17]:
valid.duplicated(subset='id').sum()

0

In [18]:
train = train.rename(columns={'clean_article': 'paragraphs', 'clean_summary': 'summary'})

In [19]:
valid = valid.rename(columns={'clean_article': 'paragraphs', 'clean_summary': 'summary'})

In [20]:
import regex as re
import unicodedata

def benerin_text(text): #Benerin text per list dari nilai kolom (belum digabung)
  
  text = unicodedata.normalize('NFKD', text)
  text = text.replace('  ', ' ')
  text = text.replace('“', '"')
  text = text.replace('‘', "'")
  text = re.sub(r' - - ', '--', text)
  text = re.sub(r'\s([.|,|?|!|:|;|=|%|$])', r'\1', text)
  text = re.sub(r'([#])\s', r'\1', text)
  text = re.sub(r'\s([/|–|-])\s', r'\1', text)  #!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
 
  text_f = re.findall(r'\(.*?\)', text) #Bakal nyari semua string yang ada di dalem "tanda" dan ngehasilin list dari stringnya
  for i in text_f:
   text = text.replace(i, f"({i[2:-2]})")

  text_g = re.findall(r'\".*?\"', text)
  for j in text_g:
   text = text.replace(j, f'"{j[2:-2]}"') 

  text_h = re.findall(r'\'.*?\'', text)
  for k in text_h:
   text = text.replace(k, f'"{k[2:-2]}"')

  text_i = re.findall(r'\[.*?\]', text)
  for h in text_i:
    text = text.replace(h, f"{h[1:-1]}")

  return text.replace('  ',' ')

In [21]:
def prepro(df):
  
  for ix in range(len(df)):
    text = []
    for i in df['paragraphs'][ix]:
      text.append(' '.join(i))
      if 'Liputan6' in text[0] and ':' in text[0]:
        val = text[0].index(':')
        text[0] = text[0][val+2::]
    df['paragraphs'][ix] = ' '.join(text)

  for ix in range(len(df)):  
    text = []
    for i in df['summary'][ix]:
      text.append(' '.join(i))
    df['summary'][ix] = ' '.join(text)

  df['paragraphs'] = df['paragraphs'].apply(lambda x: benerin_text(x).lower()) #Apply benerin_text ke column paragh per valuenya
  df['summary'] = df['summary'].apply(lambda x: benerin_text(x).lower())
  
  return df

In [22]:
train = prepro(train)
valid = prepro(valid)

/tmp/ipykernel_1333/1898393266.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['paragraphs'][ix] = ' '.join(text)
/tmp/ipykernel_1333/1898393266.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['summary'][ix] = ' '.join(text)


In [23]:
train.paragraphs[17]

'brazil menolak bergabung dalam organisasi negara-negara pengekspor minyak (opec), menyusul undangan resmi duta besar iran, moshen shaterzadeh. demikian menteri energi brazil, edson labao, seperti dilaporkan kantor berita reuters, kamis (4/9). "kami belum mempertimbangkan kemungkinannya," kata labao. pada november 2007, pemerintah brazil mengumumkan, telah menemukan ladang minyak baru di 250 km lepas pantai. tempat itu diperkirakan mengandung delapan miliar barel minyak. awal tahun ini, negara amerika selatan itu mengaku telah menemukan ladang-ladang lain, dekat ladang terdahulu, yang mengandung 40 miliar barel minyak. jika kedua ladang itu digabung, maka cadangan minyak brazil menjadi setingkat dengan nigeria. bahkan, di atas venezuela. "tawaran itu tidak disampaikan kepada para menteri brazil lain," kata labao. "dan, tidak ada diskusi mengenai penggabungan brazil dalam opec." mei lalu, kata labao, brazil diundang arab saudi untuk menghadiri pertemuan darurat opec. "pertemuan itu bert

In [24]:
train.summary[17]

'brazil menolak bergabung dalam opec, menyusul undangan resmi iran. brazil mengaku, telah menemukan ladang-ladang minyak yang mengandung 48 miliar barel minyak.'

In [25]:
train['paragraphs'].str.contains('liputan6').sum()

97

In [26]:
valid['paragraphs'].str.contains('liputan6').sum()

3

In [27]:
train = train[~train['paragraphs'].str.contains('liputan6')]

In [28]:
valid = valid[~valid['paragraphs'].str.contains('liputan6')]

In [29]:
train['paragraphs'][6997]

'satu granat, 68 amunisi jenis fn kaliber sembilan milimeter, serta satu magasin aktif ditemukan di tempat sampah di jalan ahmad yani, koperapoka, timika, papua, senin (17/9). benda berbahaya tersebut terbungkus dalam sebuah tas punggung. adalah oktovianus luturmas yang menemukan barang berbahaya tersebut. saat itu, bocah yang masih duduk di sekolah dasar itu tengah mencari kaleng bekas di tempat sampah. tiba-tiba, dia melihat satu tas punggung di tumpukan sampah. setelah dibuka ternyata tas itu berisi granat dan amunisi. oktovianus kemudian membawa tas itu ke markas kepolisian sektor mimika baru. kepala detasemen b brigade mobil timika komisaris polisi yunus wally menyatakan, semua temuan itu telah dibawa tim gegana kepolisian daerah papua ke markas komando brigade mobil (brimob) timika untuk penyidikan lebih lanjut. (ren/tim liputan 6 sctv).'

In [30]:
train['summary'][6997]

'oktovianus luturmas menemukan satu granat, 64 amunisi jenis fn kaliber sembilan milimeter, serta satu magasin di tempat sampah di timika. barang berbahaya itu saat ini berada di markas komando brimob timika.'

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14904 entries, 0 to 15000
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   paragraphs  14904 non-null  object
 1   summary     14904 non-null  object
 2   id          14904 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 981.8+ KB


In [32]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 0 to 499
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   paragraphs  497 non-null    object
 1   summary     497 non-null    object
 2   id          497 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 15.5+ KB


In [33]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train)
valid_dataset = Dataset.from_dict(valid)

In [34]:
import datasets
dd = datasets.DatasetDict({"train":train_dataset,"validation":valid_dataset})

In [35]:
dd

DatasetDict({
    train: Dataset({
        features: ['paragraphs', 'summary', 'id'],
        num_rows: 14904
    })
    validation: Dataset({
        features: ['paragraphs', 'summary', 'id'],
        num_rows: 497
    })
})

# Fine Tuning

In [36]:
from transformers import AutoConfig
conf = AutoConfig.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", no_repeat_ngram_size=3, dropout_rate=0.1, max_length=128)

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

In [37]:
conf

MT5Config {
  "_name_or_path": "csebuetnlp/mT5_multilingual_XLSum",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "length_penalty": 0.6,
  "max_length": 128,
  "model_type": "mt5",
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 250112
}

In [38]:
model_path = "google/mt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, config=conf)

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/srv/conda/envs/saturn/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [39]:
tokenizer

PreTrainedTokenizerFast(name_or_path='google/mt5-base', vocab_size=250100, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})

In [40]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (w

In [41]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [42]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["paragraphs"],
        max_length=max_input_length,
        truncation=True,
        padding=True
    )
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True, padding=True)
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels_mask"] = labels["attention_mask"]
    return model_inputs

In [43]:
preprocess_function(dd["train"][:2])

{'input_ids': [[303, 4652, 259, 273, 31614, 15449, 9102, 502, 259, 120827, 1952, 1487, 259, 50173, 1404, 269, 82999, 4481, 12230, 261, 3090, 1952, 1487, 259, 273, 31614, 3102, 259, 42386, 83884, 260, 259, 120827, 854, 259, 3193, 259, 126749, 259, 36717, 10047, 767, 13729, 15043, 262, 259, 16496, 30788, 259, 158676, 260, 62821, 647, 416, 2764, 259, 3697, 259, 42386, 83884, 259, 42444, 1952, 1487, 259, 273, 31614, 259, 4392, 259, 21603, 18042, 985, 260, 259, 218310, 35259, 854, 259, 15367, 1353, 767, 14980, 301, 120366, 259, 2145, 10047, 264, 54662, 7463, 259, 265, 22461, 260, 259, 20399, 1353, 259, 21603, 10047, 35803, 77638, 457, 1224, 20261, 693, 2336, 301, 1952, 1487, 259, 273, 31614, 3038, 847, 303, 5601, 330, 57709, 261, 259, 4073, 457, 393, 4764, 301, 6782, 270, 21063, 114781, 2777, 157187, 260, 416, 5680, 7161, 522, 331, 26437, 261, 259, 273, 31614, 3102, 259, 270, 66259, 851, 1224, 23133, 555, 9430, 259, 218310, 35259, 457, 259, 3697, 301, 65971, 303, 4652, 35238, 854, 260, 8158

In [44]:
tokenized_datasets = dd.map(preprocess_function, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [45]:
print(tokenized_datasets['train'][0])

{'paragraphs': 'sejak uefa menghapuskan ajang piala winner pada delapan tahun lalu, maka piala uefa pun naik kasta. ajang ini menjadi kompetisi tingkat klub terbesar kedua setelah liga champions. sayangnya meski sudah naik kasta nasib piala uefa hanya sedikit membaik. kejuaraan ini tetap tidak terlalu dilirik oleh klub-klub besar eropa. bahkan tidak sedikit klub raksasa yang menganggap bermain di piala uefa sama dengan sebuah bencana, seperti yang pernah dilontarkan kubu ac milan. melihat situasi demikian, uefa pun tergerak untuk mengangkat pamor kejuaraan yang sudah digelar sejak 1971 ini. salah satu upaya yang kini sedangmenjadi bahan pertimbangan uefa adalah mengganti nama kejuaraan ini. hal ini dinyatakan oleh karl-heinz rummenigge, yang dalam kapasitasnya menjadi perwakilan dari asosiasi klub-klub eropa. langkah nyata telah ditempuh oleh organisasi ini, dengan menyerahkan proposal penggantian nama tersebut pada uefa senin (07/07) ini. " harapannya adalah dengan menyegarkan nama da

In [46]:
import evaluate

rouge_score = evaluate.load("rouge")

In [47]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

In [48]:
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923,
 'rougeLsum': 0.923076923076923}

In [49]:
tokenized_datasets = tokenized_datasets.remove_columns(
    dd["train"].column_names
)

In [50]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'labels_mask'],
        num_rows: 14904
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'labels_mask'],
        num_rows: 497
    })
})

In [ ]:
'''from transformers.optimization import Adafactor, AdafactorSchedule

optimizer = Adafactor(
    model.parameters(),
    lr=1e-3,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,
)
lr_scheduler = AdafactorSchedule(optimizer)'''

In [51]:
from transformers import Seq2SeqTrainingArguments

batch_size = 2
num_train_epochs = 5
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_path.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-coba-coba-coba",
    evaluation_strategy="epoch",
    #save_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    learning_rate=5.6e-6,
    weight_decay=5e-2,
    #optim="adafactor",
    #adafactor=True,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [52]:
import numpy as np
from nltk import sent_tokenize

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [53]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [54]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  303,  4652,   259,  ..., 12111,  8925,     1],
        [  259,   263, 19083,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[   416,   2764,  15342,   1952,   1487,    259,  50173,    259,   3560,
          65233,   9102,    261,    259,  18246,    555,   9430,   1952,   1487,
            259,    273,  31614,    259,   4392,    259,  21603,    767,  23133,
            260,   1224,  35772, 117522,    647,    261,    259,    273,  31614,
            390,  47830,    851,   1224,  37189,  10553,    259, 218310,  35259,
           3653,    260,   3388,    854,    259,  15640,    847,    457,   1619,
          51536,    259,   2145,  57836,  30343,  10047,    264,  54662,    259,
            265,  22461,    260,      1,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0, 

In [55]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/jovyan/workspace/mt5-base-coba-coba-coba is already a clone of https://huggingface.co/GhifSmile/mt5-base-coba-coba-coba. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [56]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: labels_mask. If labels_mask are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/srv/conda/envs/saturn/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14904
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 37260
  Number of trainable parameters = 582401280


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,7.092200,0.653769,0.355700,0.239000,0.321600,0.334200
2,0.944200,0.690009,0.427000,0.286800,0.371000,0.402800
3,3.078900,0.677467,0.380100,0.258100,0.340000,0.356400
4,1.056500,0.592782,0.434500,0.288500,0.376000,0.410200
5,0.787200,0.586960,0.433600,0.288000,0.374600,0.409500


The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: labels_mask. If labels_mask are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 497
  Batch size = 2
Saving model checkpoint to mt5-base-coba-coba-coba/checkpoint-7452
Configuration saved in mt5-base-coba-coba-coba/checkpoint-7452/config.json
Model weights saved in mt5-base-coba-coba-coba/checkpoint-7452/pytorch_model.bin
tokenizer config file saved in mt5-base-coba-coba-coba/checkpoint-7452/tokenizer_config.json
Special tokens file saved in mt5-base-coba-coba-coba/checkpoint-7452/special_tokens_map.json
Copy vocab file to mt5-base-coba-coba-coba/checkpoint-7452/spiece.model
tokenizer config file saved in mt5-base-coba-coba-coba/tokenizer_config.json
Special tokens file saved in mt5-base-coba-coba-coba/special_tokens_map.json
Copy vocab file to mt5-base-

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: labels_mask. If labels_mask are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 497
  Batch size = 2
Saving model checkpoint to mt5-base-coba-coba-coba/checkpoint-14904
Configuration saved in mt5-base-coba-coba-coba/checkpoint-14904/config.json
Model weights saved in mt5-base-coba-coba-coba/checkpoint-14904/pytorch_model.bin
tokenizer config file saved in mt5-base-coba-coba-coba/checkpoint-14904/tokenizer_config.json
Special tokens file saved in mt5-base-coba-coba-coba/checkpoint-14904/special_tokens_map.json
Copy vocab file to mt5-base-coba-coba-coba/checkpoint-14904/spiece.model
The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: labels_mask. If labels_mask are 

TrainOutput(global_step=37260, training_loss=2.5917980937374194, metrics={'train_runtime': 27418.7557, 'train_samples_per_second': 2.718, 'train_steps_per_second': 1.359, 'total_flos': 8.935291773517824e+16, 'train_loss': 2.5917980937374194, 'epoch': 5.0})

In [57]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: labels_mask. If labels_mask are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 497
  Batch size = 2


{'eval_loss': 0.5869601368904114,
 'eval_rouge1': 0.4336,
 'eval_rouge2': 0.288,
 'eval_rougeL': 0.3746,
 'eval_rougeLsum': 0.4095,
 'eval_runtime': 734.8031,
 'eval_samples_per_second': 0.676,
 'eval_steps_per_second': 0.339,
 'epoch': 5.0}

In [58]:
trainer.push_to_hub()

Saving model checkpoint to mt5-base-coba-coba-coba
Configuration saved in mt5-base-coba-coba-coba/config.json
Model weights saved in mt5-base-coba-coba-coba/pytorch_model.bin
tokenizer config file saved in mt5-base-coba-coba-coba/tokenizer_config.json
Special tokens file saved in mt5-base-coba-coba-coba/special_tokens_map.json
Copy vocab file to mt5-base-coba-coba-coba/spiece.model


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Upload file pytorch_model.bin:   0%|          | 3.30k/2.17G [00:00<?, ?B/s]

...

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 0.4336}]}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/GhifSmile/mt5-base-coba-coba-coba
   245dff5..f31f321  main -> main

   245dff5..f31f321  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/GhifSmile/mt5-base-coba-coba-coba/commit/245dff53fea7fb9d77be18e2c3e0ec623bb4a302'